<a href="https://colab.research.google.com/github/AlfredWGA/CS6120-Final-Project/blob/main/blog_generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers sentencepiece datasets

     |████████████████████████████████| 2.1MB 7.9MB/s 
     |████████████████████████████████| 1.2MB 32.4MB/s 
     |████████████████████████████████| 225kB 50.9MB/s 
     |████████████████████████████████| 901kB 44.6MB/s 
     |████████████████████████████████| 3.3MB 52.7MB/s 
     |████████████████████████████████| 245kB 56.4MB/s 
     |████████████████████████████████| 112kB 60.7MB/s 


In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

True

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('drive/MyDrive/Kaggle/blogtext.csv')

In [ ]:
df

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...
...,...,...,...,...,...,...,...
681279,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, I could write some really ..."
681280,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, 'I have the second yeast i..."
681281,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan, Your 'boyfriend' is fuckin..."
681282,1713845,male,23,Student,Taurus,"01,July,2004","Dear Susan: Just to clarify, I am as..."


In [ ]:
df['topic'].value_counts()

indUnk                     251015
Student                    153903
Technology                  42055
Arts                        32449
Education                   29633
Communications-Media        20140
Internet                    16006
Non-Profit                  14700
Engineering                 11653
Law                          9040
Publishing                   7753
Science                      7269
Government                   6907
Consulting                   5862
Religion                     5235
Fashion                      4851
Marketing                    4769
Advertising                  4676
BusinessServices             4500
Banking                      4049
Chemicals                    3928
Telecommunications           3891
Accounting                   3832
Military                     3128
Museums-Libraries            3096
Sports-Recreation            3038
HumanResources               3010
RealEstate                   2870
Transportation               2326
Manufacturing 

In [ ]:
df['sign'].value_counts()

Cancer         65048
Aries          64979
Taurus         62561
Libra          62363
Virgo          60399
Scorpio        57161
Pisces         54053
Leo            53811
Gemini         51985
Sagittarius    50036
Aquarius       49687
Capricorn      49201
Name: sign, dtype: int64

In [ ]:
df['gender'].value_counts()

male      345193
female    336091
Name: gender, dtype: int64

In [ ]:
df['text'] = df['text'].apply(lambda x: x.strip())

In [ ]:
df['length'] = [len(x) for x in df['text']]

In [ ]:
print("Number of rows without text:", len(df[df['text'] == '']))

Number of rows without text: 3119


In [ ]:
# Drop empty text row
df = df[df['text'].str.len() > 0]

In [ ]:
print(len(df))

678165


In [ ]:
df['length']

0           137
1           160
2         25445
3            22
4           380
          ...  
681279      241
681280      377
681281       70
681282      327
681283     1247
Name: length, Length: 678165, dtype: int64

In [ ]:
df['length'].mean()

1105.8931248295032

In [ ]:
df.iloc[1000]['text']

"My son and I are visiting Mom and Dad. I have not been to their house in about a year and am overdue for a visit. It took 13 hours to drive here. Mom had dinner ready when we walked through the door. Spaghetti red, she calls it. It's pasta, hamburger, onion, and tomato sauce seasoned with chili powder. A staple from my childhood.  I came here to tell them about losing my job but find I cannot do it. My brother is still searching for work and I do not want to add to their worries.   Dad's breathing is worse. He and I went to the Walmart today and after walking from the parking lot to the store he had to sit on a bench to catch his breath. He has emphysema. His death looms just over the horizon. All the years of smoking have exacted a terrible toll and he knows it. It tragic to talk with him about the disease.  My son spent most of the day with Mom sorting quarters and filing them in albums. She is trying to get complete sets of the new state quarters for each grandchild. He was thrille

In [ ]:
df.iloc[1000]['gender']

'male'

In [ ]:
df[df['gender'] == 'female'].loc[77]['text']

"Why do I like him? He's funny We have so much in common He is so cute We have the same sick mind He likes friends (I know huh!) Unlike other guys he can actually match He is super sweet Never is he mad He is intelligent He is open, almost... ok he's blunt He is witty He is NOT gay (see, not all the good ones are) And yes ladys, he is real... and taken.  I dunno why he's with me. I didn't think he'd like a girl, well... like me. I mean, I'm  Not to bright(I almost get streight Fs) I'm fat I'm ugly I'm a bitch sometimes I'm clinicly insane (no... realy) That seems like a lot to deal with, don't you think? I don't know what the boy sees in me. But this is like a dream come true. Every girl dreams that some day the perfect guy will show up one day, even a girl like me. Well, mericals do happen. I guess there's hope after all. Even for a girl like me."

In [ ]:
import transformers
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AutoTokenizer, AutoModelWithLMHead, AutoModelForCausalLM
from datasets import Dataset

https://github.com/huggingface/notebooks/blob/master/examples/summarization.ipynb
https://github.com/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb
https://github.com/huggingface/notebooks/blob/master/examples/language_modeling.ipynb

Load data using Huggingface's `datasets` library  
https://huggingface.co/docs/datasets/loading_datasets.html

In [ ]:
print(len(df[df['gender'] == 'male']))
print(len(df[df['gender'] == 'female']))

In [ ]:
male_idx = (np.random.random_sample(size=25000) * len(df[df['gender'] == 'male'])).astype(np.int)
female_idx = (np.random.random_sample(size=25000) * len(df[df['gender'] == 'female'])).astype(np.int)

In [ ]:
df_sub = pd.concat([df[df['gender'] == 'male'].iloc[male_idx], df[df['gender'] == 'female'].iloc[female_idx]])

In [ ]:
df_sub.value_counts(['gender'])

In [ ]:
dataset = Dataset.from_pandas(df_sub)

In [ ]:
dataset

In [ ]:
dataset.features

In [ ]:
model_name = 'distilgpt2'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
# model = GPT2LMHeadModel('gpt2', pad_token_id=tokenizer.eos_token_id).to('cuda')

In [ ]:
def tokenize(sample_batch):
  return tokenizer(sample_batch['text'],
            add_special_tokens=True,
            truncation='longest_first',
            padding='max_length',
            max_length=128
            )

In [ ]:
def assign_labels(sample_batch):
  return {'labels': sample_batch['input_ids'].copy()}

In [ ]:
dataset = dataset.map(tokenize, batched=True, batch_size=3000, remove_columns=['text'])

In [ ]:
dataset = dataset.map(assign_labels, batched=True, batch_size=3000)

In [ ]:
dataset.save_to_disk('/content/drive/MyDrive/CS 6120 - Final Project/dataset_pad')
# dataset.save_to_disk('/content/drive/MyDrive/CS 6120 - Final Project/dataset_no_pad')

In [ ]:
# from datasets import load_from_disk
# dataset = load_from_disk('/content/drive/MyDrive/CS 6120 - Final Project/dataset_pad')

In [ ]:
dataset = dataset.train_test_split(test_size=0.1)

In [ ]:
dataset

In [ ]:
len(dataset['train'][0]['input_ids'])

In [ ]:
tokenizer.decode(dataset['train'][100]['input_ids'], skip_special_tokens=True)

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModelForCausalLM

In [ ]:
from transformers import Trainer, TrainingArguments

https://github.com/huggingface/blog/blob/master/notebooks/02_how_to_generate.ipynb

Please referto https://github.com/huggingface/transformers/issues/1413 on adding new tokens.

In [ ]:
len(tokenizer)

50257

In [ ]:
tokenizer.special_tokens_map

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'pad_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>'}

In [ ]:
tokenizer.add_tokens([f'<|{x}|>' for x in df['gender'].unique()])
print(len(tokenizer))

50259


In [ ]:
model.resize_token_embeddings(len(tokenizer))

NameError: ignored

In [ ]:
def add_hint_tokens(sample_batch):
  input_ids = []
  for ids in sample_batch['input_ids']:
    input_ids.append(tokenizer.convert_tokens_to_ids(['<|male|>']) + ids)
  attention_mask = [[1] + x for x in sample_batch['attention_mask']]
  return {'input_ids': input_ids, 'attention_mask': attention_mask}

In [ ]:
dataset = dataset.map(add_hint_tokens, batched=True, batch_size=3000)

In [ ]:
dataset = dataset.map(assign_labels, batched=True, batch_size=3000)

In [ ]:
print(tokenizer.decode(dataset['train'][0]['input_ids'], skip_special_tokens=True))
print(tokenizer.decode(dataset['train'][0]['labels'], skip_special_tokens=True))

<|male|>                 i hate eavesdroppers. they are the ones who listen secretly to a private conversation. instead of sneaking and listening, go ask the person about it if u wanna know the real story. eavesdroppin is bad. if u don't believe me, ask GOD! Ü               
<|male|>                 i hate eavesdroppers. they are the ones who listen secretly to a private conversation. instead of sneaking and listening, go ask the person about it if u wanna know the real story. eavesdroppin is bad. if u don't believe me, ask GOD! Ü               


In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda')

In [ ]:
training_args = TrainingArguments(
    "/content/drive/MyDrive/CS 6120 - Final Project/test-clm/with-hint",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=48,
    save_strategy='epoch',
    logging_strategy='epoch',
    save_total_limit=1
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,3.339600,2.846267,22.088000,226.367000
2,2.881200,2.827590,22.000700,227.265000
3,2.857500,2.822855,22.044800,226.811000


TrainOutput(global_step=2814, training_loss=3.026074288157427, metrics={'train_runtime': 1782.9907, 'train_samples_per_second': 1.578, 'total_flos': 8559205562880000.0, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -341295104, 'train_mem_gpu_alloc_delta': 983266304, 'train_mem_cpu_peaked_delta': 351752192, 'train_mem_gpu_peaked_delta': 10574880256})

In [ ]:
# model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/CS 6120 - Final Project/test-clm/with-hint/checkpoint-2814", pad_token_id=tokenizer.eos_token_id).to('cuda')
model = AutoModelForCausalLM.from_pretrained(model_name, pad_token_id=tokenizer.eos_token_id).to('cuda')

In [ ]:
input_sentence = "Why do I like him? He's funny We and have so much in common."
sample_inpud_ids = tokenizer.encode(input_sentence, return_tens
                                    ors='pt').cuda()
sample_output = model.generate(
    sample_inpud_ids, 
    do_sample=True, 
    max_length=128,
    top_p=0.9,
    top_k=50,
    no_repeat_ngram_size=3,
    # temperature=0.5,
    num_return_sequences=10
)
print("Input:\n" + 100 * '-')
print(input_sentence)
print("Output:\n" + 100 * '-')
for i, s in enumerate(sample_output):
  print(f'{i+1}.', tokenizer.decode(s, skip_special_tokens=True))

Input:
----------------------------------------------------------------------------------------------------
Why do I like him? He's funny We and have so much in common.
Output:
----------------------------------------------------------------------------------------------------
1. Why do I like him? He's funny We and have so much in common. I don't like the idea of a character as a human, but I like the concept of a hero being that he's kind of a little, just like a big boy who's been through the pain of a lifetime, you know.


"I like that guy, just because he's just one of the best characters ever. I like that. I think he's the best character ever. He's one of those guys. He could be a real hero. He just keeps me from being the best person ever, not because he hasn
2. Why do I like him? He's funny We and have so much in common. He does, in a very emotional way, just a little bit of a joy. I wish he had some time in the car. But if I was to have any kind of love to the guy, it would be

In [ ]:
input_sentence = "<|male|> Why do I like him? He's funny We and have so much in common."
sample_inpud_ids = tokenizer.encode(input_sentence, return_tensors='pt').cuda() 

In [ ]:
sample_output = model.generate(
    sample_inpud_ids, 
    do_sample=True, 
    max_length=100,
    top_p=0.95,
    # top_k=0,
    no_repeat_ngram_size=4,
    temperature=0.8
)
print("Input:\n" + 100 * '-')
print(input_sentence)
print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input:
----------------------------------------------------------------------------------------------------
<|male|> Why do I like him? He's funny We and have so much in common.
Output:
----------------------------------------------------------------------------------------------------
<|male|> Why do I like him? He's funny We and have so much in common.    


In [ ]:
input_sentence = '<|male|> I fall in love with a'
sample_inpud_ids = tokenizer.encode(input_sentence, return_tensors='pt').cuda() 

In [ ]:
sample_output = model.generate(
    sample_inpud_ids, 
    do_sample=True, 
    max_length=100, 
    top_p=0.95,
    no_repeat_ngram_size=4,
    temperature=0.9,
    num_return_sequences=10
)
print("Input:\n" + 100 * '-')
print(input_sentence)
print("Output:\n" + 100 * '-')
for i in range(len(sample_output)):
  print(tokenizer.decode(sample_output[i], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input:
----------------------------------------------------------------------------------------------------
<|male|> I fall in love with a
Output:
----------------------------------------------------------------------------------------------------
<|male|> I fall in love with a man who's a good person, he wants to see you. I fall inlove with a man in a bad way. That's the way you do it. I fall into love with a woman that really matters. I fall out of love with a guy who makes me love you more than you have. He loves me more than you. His friend is his friend. He loves him more than you do. And he loves me more.
<|male|> I fall in love with a woman who is very good at telling stories about her life,””    
<|male|> I fall in love with a guy called Daphne.”
<|male|> I fall in love with a woman who is pretty, I'm in love with her.     *hugs*   
<|male|> I fall in love with a cat. It's just that it doesn't care.    
<|male|> I fall in love with a certain amount of people.”    
<|male|> I fa

In [ ]:
input_sentence = '<|female|> I fall in love with a'
sample_inpud_ids = tokenizer.encode(input_sentence, return_tensors='pt').cuda() 

In [ ]:
sample_output = model.generate(
    sample_inpud_ids, 
    do_sample=True, 
    max_length=128, 
    top_p=0.95,
    no_repeat_ngram_size=4,
    temperature=0.9,
    num_return_sequences=10
)
print("Input:\n" + 100 * '-')
print(input_sentence)
print("Output:\n" + 100 * '-')
for i in range(len(sample_output)):
  print(tokenizer.decode(sample_output[i], skip_special_tokens=True))

Input:
----------------------------------------------------------------------------------------------------
<|male|> Yesterday I told you about my progression into Hippy-ness, but there was been some developments in the time sinse I have posted      Back to the topic of me being a Hippy.  After the events of last night it seems I have a hippy alter ego, like Yugi has an alter ego who is an ultra cool, slightly taller, card playing ancient pharaoh. Sadly my alter ego isn’t ultra cool or slightly taller or has his own television show and range of merchandise. No my alter ego seems to be the unholy child of Beck (the much loved 70’s styled artist of the mid 90’s) and Shaggy (from Scooby doo)  I think I may gain some amazing Hippy Powers when I metamorphose but all I can remember was the uncontrollable desire for Pizza, to have my hair put into dread locks and more importantly using the words ‘like’, ‘man’ and ‘dude’ far to much.  As I was saying last night was a blur, but when I woke up t

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
df_gen = {'gender': [], 'text': []}
n_gen = 2000
male_idx = (np.random.random_sample(size=int(n_gen/2)) * len(df[df['gender'] == 'male'])).astype(np.int)
female_idx = (np.random.random_sample(size=int(n_gen/2)) * len(df[df['gender'] == 'female'])).astype(np.int)
df_sub = pd.concat([df[df['gender'] == 'male'].iloc[male_idx], df[df['gender'] == 'female'].iloc[female_idx]])
for row in tqdm(df_sub[['gender', 'text']].itertuples(), total=n_gen):
  input_sentence = f'<|{row.gender}|> ' + row.text
  sample_inpud_ids = tokenizer.encode(input_sentence, truncation=True, max_length=32, return_tensors='pt').cuda()

  sample_output = model.generate(
      sample_inpud_ids, 
      do_sample=True,
      max_length=128, 
      top_p=0.95,
      no_repeat_ngram_size=4,
      temperature=0.9,
      num_return_sequences=20
  )
  for s in sample_output:
    # The first token is <|gender|>
    text = tokenizer.decode(s, skip_special_tokens=True)
    special_end = text.find('>')
    # df_gen['text'].append(text)
    df_gen['text'].append(text[special_end+1:])
    df_gen['gender'].append(row.gender)

In [ ]:
df_gen = pd.DataFrame(df_gen)

In [ ]:
df_gen

,gender,text
0,male,"HEYA all. I'm in Spring Day, and the practices..."
1,male,"HEYA all. I'm in Spring Day, and the practices..."
2,male,"HEYA all. I'm in Spring Day, and the practices..."
3,male,"HEYA all. I'm in Spring Day, and the practices..."
4,male,"HEYA all. I'm in Spring Day, and the practices..."
...,...,...
39995,female,I lay my life before you/and i'm not getting u...
39996,female,I lay my life before you/and i'm not getting u...
39997,female,I lay my life before you/and i'm not getting u...
39998,female,I lay my life before you/and i'm not getting u...


In [ ]:
df_gen.iloc[0]['text'][:100]

" HEYA all. I'm in Spring Day, and the practices were yesterday...well, one of them.  You should hear"

In [ ]:
df_gen['text'] = df_gen['text'].apply(lambda x: x.strip())

In [ ]:
df_gen.to_csv("/content/drive/MyDrive/CS 6120 - Final Project/blog_gen.csv", index=False)

In [ ]:
from tqdm.notebook import tqdm
df_gen_naive = {'gender': [], 'text': []}
n_gen = 4000
df_male = df[df['gender'] == 'male'].sample(n=int(n_gen/2))
df_female = df[df['gender'] == 'female'].sample(n=int(n_gen/2))
df_sub = pd.concat([df_male, df_female])
for row in tqdm(df_sub[['gender', 'text']].itertuples(), total=len(df_male) + len(df_female)):
  input_sentence = row.text
  sample_inpud_ids = tokenizer.encode(input_sentence, truncation=True, max_length=32, return_tensors='pt').cuda()
  try:
    sample_output = model.generate(
        sample_inpud_ids, 
        do_sample=True,
        max_length=128, 
        top_p=0.9,
        top_k=50,
        no_repeat_ngram_size=3,
        num_return_sequences=10
    )
  except:
    print('Something happens. Pass.')
  for s in sample_output:
    text = tokenizer.decode(s, skip_special_tokens=True)
    df_gen_naive['text'].append(text)
    df_gen_naive['gender'].append(row.gender)

In [ ]:
df_gen_naive = pd.DataFrame(df_gen_naive)
df_gen_naive['text'] = df_gen_naive['text'].apply(lambda x: x.strip())

In [ ]:
df_gen_naive

,gender,text
0,male,Me Am Ben. I spend many hours a day sitting st...
1,male,Me Am Ben. I spend many hours a day sitting st...
2,male,Me Am Ben. I spend many hours a day sitting st...
3,male,Me Am Ben. I spend many hours a day sitting st...
4,male,Me Am Ben. I spend many hours a day sitting st...
...,...,...
39995,female,well i woke up and talked to CAG for a while t...
39996,female,well i woke up and talked to CAG for a while t...
39997,female,well i woke up and talked to CAG for a while t...
39998,female,well i woke up and talked to CAG for a while t...


In [ ]:
df_gen_naive.to_csv("/content/drive/MyDrive/CS 6120 - Final Project/blog_gen_naive.csv", index=False)